In [1]:
import json
import jieba
import re
import numpy as np
import pandas as pd
from zhconv import convert
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

In [2]:
# 这是用来生成lsa向量的训练集，必须用这个数据集来拼接lsa向量
test_data_lsa_cvjd_13586 = pd.read_pickle('../generalization_data/test_data_lsa_cvjd_13586.pkl')
test_data_lsa_cvjd_13586

,cv_id,jd_id,jd_code,title,deadline,category_id,category_name,company_id,company_name,recruit_number,...,workYear,avatar,eduTracks,jobTracks,projectTracks,ds.1,da_label,rank_label,resume_type,order_month
1,2c9207157bf91042017c0d979cbb047d,2c9207157bcbb76b017bf7f2bbbb3153,JD20210918160449291911,会员管理经理,2021-10-31 23:59:59.0,130101,用户运营,2c9207157b9b9199017bc895b2f23a39,上海晨光文具股份有限公司,1,...,7.0,NaN,"['{""degree"": ""本科"", ""endDate"": ""2014-06"", ""id"":...","['{""companyName"": ""北京美餐好客科技有限公司"", ""companyNatu...","['{""description"": ""项目描述: 母婴类人群，包含童装、童鞋、奶粉。\\n运...",20220831,1,1,NaN,2021-09
2,2c9207157bf91042017c7232fbab3be1,2c9207157bf94d99017c108f769405e0,JD20210923104433723714,产品市场经理,2021-10-31 23:59:59.0,110101,产品经理,2c9207157bf94d99017c108765c804aa,凯斯纽荷兰（中国）管理有限公司,1,...,14.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2012-09"", ""id"":...","['{""companyName"": ""Continental Tires (China) L...",NaN,20220831,2,2,NaN,2021-10
9,2c9207157dbedf8d017dc7561a247ced,2c9207157dbedfdb017dc10786e9259d,JD20211216101338040534,专卖店店长,NaN,290304,门店店长,2c9207157b9b9199017b9f3db5430383,小米科技有限责任公司,1,...,6.0,NaN,"['{""degree"": ""大专"", ""endDate"": ""2015-06"", ""id"":...","['{""companyName"": ""广州金逸影视传媒股份有限公司"", ""descripti...","['{""companyName"": ""*霍尔果斯自由时光影院管理有限公司"", ""descri...",20220831,3,3,4.0,2022-03
15,2c9207157e58b769017e61a3e13c0944,2c9207157e425d5c017e48061bb50e14,JD20220111152353418578,Java开发工程师,NaN,100101,Java,2c9207157bf94d99017c2f902e952262,深圳纳欣科技有限公司,1,...,NaN,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2011-06"", ""id"":...","['{""companyName"": ""任职时间"", ""endDate"": ""9999-02""...","['{""description"": ""公司名称\\n盛彩科技信息服务有限公司\\n职位名称\...",20220831,1,1,NaN,2022-01
27,2c9207157e58b769017e9a416cff07d9,2c9207157e425d5c017e47121b3a6eef,JD20220111102941471259,APP主UI设计,NaN,120105,UI设计师,2c9207157dabb0c9017db32ad4b5182e,北京能量盒科技有限公司,1,...,8.0,NaN,"['{""degree"": ""本科"", ""endDate"": ""2014-07"", ""id"":...","['{""companyName"": ""北京默契破冰科技有限公司"", ""description...","['{""companyName"": ""*北京默契破冰科技有限公司"", ""descriptio...",20220831,1,1,4.0,2022-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82301,8a69c4b5820f0b2101820f99257c07b7,2c9207158114fb2901811ee8a68f49ea,JD20220531210148031570,人力负责人,NaN,150406,人力资源VP/CHO,2c920715800e886301805e59ca756861,马上消费金融股份有限公司,1,...,15.0,NaN,"['{""schoolTag"": ""985 211"", ""endDate"": ""2020-04...","['{""endDate"": ""2022-07"", ""companyName"": ""德坤供应链...","['{""endDate"": ""2020-08"", ""companyName"": ""德坤"", ...",20220831,1,1,NaN,2022-07
82314,8a69d7a68219933d01823fff37d04ba3,8a69f6f8823a93b901823ffcfae57267,JD20220208163227909229,合规总监,NaN,180302,法务,2c9207157ed86c8e017ed86eb428012b,东方财富信息股份有限公司,1,...,14.0,NaN,"['{""endDate"": ""2014-01"", ""unified"": true, ""deg...","['{""endDate"": ""2022-07"", ""companyName"": ""CNCB ...","['{""name"": null}']",20220831,2,2,4.0,2022-07
82318,8a69d7c282199453018233eef4a43340,2c92071581ebf3390181f5b6628042bd,JD20220713115939586417,交付经理,NaN,300701,其他生产制造职位,2c920715816798cb01816a305e95331b,广州视源电子科技股份有限公司,1,...,7.0,NaN,"['{""endDate"": ""2015-06"", ""unified"": true, ""deg...","['{""endDate"": ""2022-07"", ""companyName"": ""深南电路股...","['{""name"": null}']",20220831,3,3,4.0,2022-07
82319,8a69d7c2821994530182395f8b7032c7,8a69f68e82198e91018234ee6739094a,JD20220725183354121210,跨境电商运营经理,NaN,130124,跨境电商运营,2c9207158114fb29018132ace5f11fe9,广州视源电子科技股份有限公司,1,...,6.0,NaN,"['{""endDate"": ""2015-06"", ""unified"": true, ""deg...","['{""endDate"": ""2022-07"", ""companyName"": ""广州拉丁互...","['{""endDate"": ""2020-12"", ""jobTitle"": ""项目负责人"", ...",20220831,1,1,4.0,2022-07


In [3]:
test_data_lsa_cvjd_13586.reset_index(drop=True)

,cv_id,jd_id,jd_code,title,deadline,category_id,category_name,company_id,company_name,recruit_number,...,workYear,avatar,eduTracks,jobTracks,projectTracks,ds.1,da_label,rank_label,resume_type,order_month
0,2c9207157bf91042017c0d979cbb047d,2c9207157bcbb76b017bf7f2bbbb3153,JD20210918160449291911,会员管理经理,2021-10-31 23:59:59.0,130101,用户运营,2c9207157b9b9199017bc895b2f23a39,上海晨光文具股份有限公司,1,...,7.0,NaN,"['{""degree"": ""本科"", ""endDate"": ""2014-06"", ""id"":...","['{""companyName"": ""北京美餐好客科技有限公司"", ""companyNatu...","['{""description"": ""项目描述: 母婴类人群，包含童装、童鞋、奶粉。\\n运...",20220831,1,1,NaN,2021-09
1,2c9207157bf91042017c7232fbab3be1,2c9207157bf94d99017c108f769405e0,JD20210923104433723714,产品市场经理,2021-10-31 23:59:59.0,110101,产品经理,2c9207157bf94d99017c108765c804aa,凯斯纽荷兰（中国）管理有限公司,1,...,14.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2012-09"", ""id"":...","['{""companyName"": ""Continental Tires (China) L...",NaN,20220831,2,2,NaN,2021-10
2,2c9207157dbedf8d017dc7561a247ced,2c9207157dbedfdb017dc10786e9259d,JD20211216101338040534,专卖店店长,NaN,290304,门店店长,2c9207157b9b9199017b9f3db5430383,小米科技有限责任公司,1,...,6.0,NaN,"['{""degree"": ""大专"", ""endDate"": ""2015-06"", ""id"":...","['{""companyName"": ""广州金逸影视传媒股份有限公司"", ""descripti...","['{""companyName"": ""*霍尔果斯自由时光影院管理有限公司"", ""descri...",20220831,3,3,4.0,2022-03
3,2c9207157e58b769017e61a3e13c0944,2c9207157e425d5c017e48061bb50e14,JD20220111152353418578,Java开发工程师,NaN,100101,Java,2c9207157bf94d99017c2f902e952262,深圳纳欣科技有限公司,1,...,NaN,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2011-06"", ""id"":...","['{""companyName"": ""任职时间"", ""endDate"": ""9999-02""...","['{""description"": ""公司名称\\n盛彩科技信息服务有限公司\\n职位名称\...",20220831,1,1,NaN,2022-01
4,2c9207157e58b769017e9a416cff07d9,2c9207157e425d5c017e47121b3a6eef,JD20220111102941471259,APP主UI设计,NaN,120105,UI设计师,2c9207157dabb0c9017db32ad4b5182e,北京能量盒科技有限公司,1,...,8.0,NaN,"['{""degree"": ""本科"", ""endDate"": ""2014-07"", ""id"":...","['{""companyName"": ""北京默契破冰科技有限公司"", ""description...","['{""companyName"": ""*北京默契破冰科技有限公司"", ""descriptio...",20220831,1,1,4.0,2022-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13581,8a69c4b5820f0b2101820f99257c07b7,2c9207158114fb2901811ee8a68f49ea,JD20220531210148031570,人力负责人,NaN,150406,人力资源VP/CHO,2c920715800e886301805e59ca756861,马上消费金融股份有限公司,1,...,15.0,NaN,"['{""schoolTag"": ""985 211"", ""endDate"": ""2020-04...","['{""endDate"": ""2022-07"", ""companyName"": ""德坤供应链...","['{""endDate"": ""2020-08"", ""companyName"": ""德坤"", ...",20220831,1,1,NaN,2022-07
13582,8a69d7a68219933d01823fff37d04ba3,8a69f6f8823a93b901823ffcfae57267,JD20220208163227909229,合规总监,NaN,180302,法务,2c9207157ed86c8e017ed86eb428012b,东方财富信息股份有限公司,1,...,14.0,NaN,"['{""endDate"": ""2014-01"", ""unified"": true, ""deg...","['{""endDate"": ""2022-07"", ""companyName"": ""CNCB ...","['{""name"": null}']",20220831,2,2,4.0,2022-07
13583,8a69d7c282199453018233eef4a43340,2c92071581ebf3390181f5b6628042bd,JD20220713115939586417,交付经理,NaN,300701,其他生产制造职位,2c920715816798cb01816a305e95331b,广州视源电子科技股份有限公司,1,...,7.0,NaN,"['{""endDate"": ""2015-06"", ""unified"": true, ""deg...","['{""endDate"": ""2022-07"", ""companyName"": ""深南电路股...","['{""name"": null}']",20220831,3,3,4.0,2022-07
13584,8a69d7c2821994530182395f8b7032c7,8a69f68e82198e91018234ee6739094a,JD20220725183354121210,跨境电商运营经理,NaN,130124,跨境电商运营,2c9207158114fb29018132ace5f11fe9,广州视源电子科技股份有限公司,1,...,6.0,NaN,"['{""endDate"": ""2015-06"", ""unified"": true, ""deg...","['{""endDate"": ""2022-07"", ""companyName"": ""广州拉丁互...","['{""endDate"": ""2020-12"", ""jobTitle"": ""项目负责人"", ...",20220831,1,1,4.0,2022-07


In [4]:
pd.read_pickle('../generalization_data/test_description_tfidf_lsa.pkl')

,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,69
0,0.178965,0.075714,-0.109266,-0.020789,0.090811,0.083465,-0.115410,0.062282,-0.065541,0.080335,...,-0.024228,0.015502,0.005782,-0.036866,0.022401,0.024743,0.006560,0.033953,0.026720,0.005745
1,0.002254,0.001350,0.000630,0.010316,0.002397,-0.002676,0.016715,0.017740,0.466535,0.393983,...,-0.003291,0.001483,-0.000791,0.002276,0.000377,0.001218,-0.000770,-0.001105,-0.000217,-0.000341
2,0.191609,0.043307,-0.045695,0.118321,-0.056403,0.015801,-0.086036,0.043747,0.032610,-0.043113,...,0.089347,0.063964,0.046093,-0.020381,-0.056675,0.009606,0.030732,-0.061735,-0.028639,0.012358
3,0.335269,-0.061934,0.284877,0.042728,0.169813,-0.028494,-0.148350,-0.103335,-0.172357,0.210863,...,0.068587,0.044540,0.036146,-0.107403,0.175346,-0.118080,-0.103516,0.054410,0.056851,-0.085562
4,0.193992,-0.128040,-0.063184,-0.038710,-0.080186,-0.012763,0.030280,-0.048730,0.031442,-0.033434,...,0.014973,0.011046,0.013229,-0.074957,-0.003898,0.057383,0.044206,-0.037283,0.011132,0.004109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13581,0.188945,0.304925,0.126141,-0.202136,-0.130036,-0.048007,0.040409,0.036299,0.010519,-0.016480,...,0.006152,0.093718,-0.016312,-0.082234,0.103054,0.102797,-0.040001,0.011106,0.077187,0.000808
13582,0.085407,0.046407,0.036876,0.026928,0.011504,0.017880,-0.051051,0.004241,0.051204,-0.061756,...,0.001252,-0.013241,-0.020919,-0.032501,0.005756,-0.024688,-0.000606,0.009736,-0.004670,0.020377
13583,0.155787,0.074733,-0.022691,0.094083,-0.002425,0.028647,-0.007751,-0.050581,0.024363,-0.020570,...,0.040844,-0.015285,-0.025511,0.036116,0.011085,0.010958,-0.013619,0.029751,0.008004,0.044847
13584,0.198757,0.072944,-0.127373,0.154800,0.024872,0.022062,-0.029636,0.037140,-0.020943,0.010289,...,-0.030726,-0.013019,-0.012965,-0.007778,0.025559,-0.003695,0.025216,-0.015497,0.004063,-0.030768


# tfidf_lsa

In [5]:
# 读取生成的5份tfidf_pca文件，并拼接
# 注意，不是join的时候，concat是通过index拼接的。所以需要将索引重置。
sample = test_data_lsa_cvjd_13586.reset_index(drop=True)

# 需要将这个向量变成一个列表
def tfidf_lsa_merge_fun(series):
    return list(series)

tfidf_lsa_file = ['test_description_tfidf_lsa.pkl', 'test_position_tfidf_lsa.pkl', 'test_requirement_tfidf_lsa.pkl', 'test_skills_tfidf_lsa.pkl', 'test_title_category_tags_tfidf_lsa.pkl']
for pkl_file in tfidf_lsa_file:
    tfidf_lsa = pd.read_pickle('../generalization_data/'+pkl_file)
    print(pkl_file)
    #print(tfidf_pca)
    col_name = pkl_file.strip('.pkl') # 把skills的s去掉了
    tfidf_lsa[col_name] = tfidf_lsa.apply(tfidf_lsa_merge_fun, axis=1)
    sample = pd.concat([sample, tfidf_lsa[[col_name]]], axis=1)

tfidf_lsa_col_names = [pkl_file.strip('.pkl') for pkl_file in tfidf_lsa_file]
sample[['cv_id', 'jd_id']+tfidf_lsa_col_names]

test_description_tfidf_lsa.pkl
test_position_tfidf_lsa.pkl
test_requirement_tfidf_lsa.pkl
test_skills_tfidf_lsa.pkl
test_title_category_tags_tfidf_lsa.pkl


,cv_id,jd_id,test_description_tfidf_lsa,test_position_tfidf_lsa,test_requirement_tfidf_lsa,test_skills_tfidf_lsa,test_title_category_tags_tfidf_lsa
0,2c9207157bf91042017c0d979cbb047d,2c9207157bcbb76b017bf7f2bbbb3153,"[0.17896497513724188, 0.07571414550659793, -0....","[0.171817730230271, -0.035070685612599815, -0....","[0.04458234091145562, -0.03407758459309192, 0....","[0.3582266403278269, -0.17046381930752355, 0.2...","[0.3252484303865592, -0.1404601755384386, -0.1..."
1,2c9207157bf91042017c7232fbab3be1,2c9207157bf94d99017c108f769405e0,"[0.0022543548572903855, 0.0013497502728216206,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[7.362506665850695e-05, -1.7992806189079e-05, ...","[0.025060678965842003, 0.008574558405555977, -...","[0.3068414446421549, -0.0731466854560243, 0.34..."
2,2c9207157dbedf8d017dc7561a247ced,2c9207157dbedfdb017dc10786e9259d,"[0.19160914563647896, 0.043307168218026076, -0...","[0.0006823661944716316, -0.0001667280999103190...","[0.12353797170875343, -0.10053731489253163, 0....","[0.35214530199012284, -0.2733977116285107, -0....","[0.16655266271797456, -0.05843355822813081, -0..."
3,2c9207157e58b769017e61a3e13c0944,2c9207157e425d5c017e48061bb50e14,"[0.3352685938668621, -0.06193383868396666, 0.2...","[0.06824288797273276, 0.3345999082804966, -0.0...","[0.31934960678804675, 0.33476821858920475, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.21647338719008005, 0.6192388484570416, -0.0..."
4,2c9207157e58b769017e9a416cff07d9,2c9207157e425d5c017e47121b3a6eef,"[0.193992358450605, -0.12803979466841925, -0.0...","[0.005677395289748753, 0.003916162454285459, 0...","[0.21814583275640784, -0.01721680318513782, -0...","[0.26224908552086346, 0.10211601798150097, 0.1...","[0.08803453540478255, 0.03652972730097373, 0.0..."
...,...,...,...,...,...,...,...
13581,8a69c4b5820f0b2101820f99257c07b7,2c9207158114fb2901811ee8a68f49ea,"[0.18894454111724257, 0.30492528922333145, 0.1...","[0.04997392570055781, -0.004502174307944112, 0...","[0.20130461498046187, -0.12255278588074696, 0....","[0.4372346675297544, -0.23536595083043993, -0....","[0.09944266033041682, 0.015459251512970253, -0..."
13582,8a69d7a68219933d01823fff37d04ba3,8a69f6f8823a93b901823ffcfae57267,"[0.08540682135035549, 0.0464066016768764, 0.03...","[0.03691517113152803, -0.0035258117015806523, ...","[0.23004562348177082, -0.09744323796595142, 0....","[0.02841189931214894, 0.02501481787894325, 0.0...","[0.08820578402787298, -0.01840399298441212, -0..."
13583,8a69d7c282199453018233eef4a43340,2c92071581ebf3390181f5b6628042bd,"[0.1557873230952529, 0.07473260131093742, -0.0...","[0.07266604344946537, -0.009615434260558572, 0...","[0.23905501196703516, -0.07952094007708325, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.12134680786849046, 0.0131295908164011, 0.03..."
13584,8a69d7c2821994530182395f8b7032c7,8a69f68e82198e91018234ee6739094a,"[0.1987571952893996, 0.07294446763122266, -0.1...","[0.271392628512948, -0.05242300248651601, 0.50...","[0.06003737476583179, -0.05225116621866782, -0...","[0.33075036193849955, -0.12686838157002917, 0....","[0.30693235404040736, -0.13829145445971983, -0..."


In [6]:
sample[['cv_id', 'jd_id']+tfidf_lsa_col_names].to_pickle('../generalization_data/test_data_lsa_cvjd_merge_13586.pkl')

In [7]:
cvjd_lsa = sample[['cv_id', 'jd_id']+tfidf_lsa_col_names]
cvjd_lsa

,cv_id,jd_id,test_description_tfidf_lsa,test_position_tfidf_lsa,test_requirement_tfidf_lsa,test_skills_tfidf_lsa,test_title_category_tags_tfidf_lsa
0,2c9207157bf91042017c0d979cbb047d,2c9207157bcbb76b017bf7f2bbbb3153,"[0.17896497513724188, 0.07571414550659793, -0....","[0.171817730230271, -0.035070685612599815, -0....","[0.04458234091145562, -0.03407758459309192, 0....","[0.3582266403278269, -0.17046381930752355, 0.2...","[0.3252484303865592, -0.1404601755384386, -0.1..."
1,2c9207157bf91042017c7232fbab3be1,2c9207157bf94d99017c108f769405e0,"[0.0022543548572903855, 0.0013497502728216206,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[7.362506665850695e-05, -1.7992806189079e-05, ...","[0.025060678965842003, 0.008574558405555977, -...","[0.3068414446421549, -0.0731466854560243, 0.34..."
2,2c9207157dbedf8d017dc7561a247ced,2c9207157dbedfdb017dc10786e9259d,"[0.19160914563647896, 0.043307168218026076, -0...","[0.0006823661944716316, -0.0001667280999103190...","[0.12353797170875343, -0.10053731489253163, 0....","[0.35214530199012284, -0.2733977116285107, -0....","[0.16655266271797456, -0.05843355822813081, -0..."
3,2c9207157e58b769017e61a3e13c0944,2c9207157e425d5c017e48061bb50e14,"[0.3352685938668621, -0.06193383868396666, 0.2...","[0.06824288797273276, 0.3345999082804966, -0.0...","[0.31934960678804675, 0.33476821858920475, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.21647338719008005, 0.6192388484570416, -0.0..."
4,2c9207157e58b769017e9a416cff07d9,2c9207157e425d5c017e47121b3a6eef,"[0.193992358450605, -0.12803979466841925, -0.0...","[0.005677395289748753, 0.003916162454285459, 0...","[0.21814583275640784, -0.01721680318513782, -0...","[0.26224908552086346, 0.10211601798150097, 0.1...","[0.08803453540478255, 0.03652972730097373, 0.0..."
...,...,...,...,...,...,...,...
13581,8a69c4b5820f0b2101820f99257c07b7,2c9207158114fb2901811ee8a68f49ea,"[0.18894454111724257, 0.30492528922333145, 0.1...","[0.04997392570055781, -0.004502174307944112, 0...","[0.20130461498046187, -0.12255278588074696, 0....","[0.4372346675297544, -0.23536595083043993, -0....","[0.09944266033041682, 0.015459251512970253, -0..."
13582,8a69d7a68219933d01823fff37d04ba3,8a69f6f8823a93b901823ffcfae57267,"[0.08540682135035549, 0.0464066016768764, 0.03...","[0.03691517113152803, -0.0035258117015806523, ...","[0.23004562348177082, -0.09744323796595142, 0....","[0.02841189931214894, 0.02501481787894325, 0.0...","[0.08820578402787298, -0.01840399298441212, -0..."
13583,8a69d7c282199453018233eef4a43340,2c92071581ebf3390181f5b6628042bd,"[0.1557873230952529, 0.07473260131093742, -0.0...","[0.07266604344946537, -0.009615434260558572, 0...","[0.23905501196703516, -0.07952094007708325, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.12134680786849046, 0.0131295908164011, 0.03..."
13584,8a69d7c2821994530182395f8b7032c7,8a69f68e82198e91018234ee6739094a,"[0.1987571952893996, 0.07294446763122266, -0.1...","[0.271392628512948, -0.05242300248651601, 0.50...","[0.06003737476583179, -0.05225116621866782, -0...","[0.33075036193849955, -0.12686838157002917, 0....","[0.30693235404040736, -0.13829145445971983, -0..."


# work_duration equal_job salary

In [8]:
work_duration_mean_df = pd.read_pickle('../generalization_data/test_work_duration_mean.pkl')
equal_job_df = pd.read_pickle('../generalization_data/test_equal_job.pkl')
salary_df = pd.read_pickle('../generalization_data/test_salary.pkl')

In [9]:
cvjd_lsa_work_duration_equal_job_salary = cvjd_lsa.\
    join(work_duration_mean_df.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner').\
    join(equal_job_df.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner').\
    join(salary_df.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner')

cvjd_lsa_work_duration_equal_job_salary

,cv_id,jd_id,test_description_tfidf_lsa,test_position_tfidf_lsa,test_requirement_tfidf_lsa,test_skills_tfidf_lsa,test_title_category_tags_tfidf_lsa,work_duration_mean,equal_job,parsed_desiredSalary,parsed_currentSalary
0,2c9207157bf91042017c0d979cbb047d,2c9207157bcbb76b017bf7f2bbbb3153,"[0.17896497513724188, 0.07571414550659793, -0....","[0.171817730230271, -0.035070685612599815, -0....","[0.04458234091145562, -0.03407758459309192, 0....","[0.3582266403278269, -0.17046381930752355, 0.2...","[0.3252484303865592, -0.1404601755384386, -0.1...",506.000000,4,-1,-1
1,2c9207157bf91042017c7232fbab3be1,2c9207157bf94d99017c108f769405e0,"[0.0022543548572903855, 0.0013497502728216206,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[7.362506665850695e-05, -1.7992806189079e-05, ...","[0.025060678965842003, 0.008574558405555977, -...","[0.3068414446421549, -0.0731466854560243, 0.34...",811.500000,0,-1,-1
2,2c9207157dbedf8d017dc7561a247ced,2c9207157dbedfdb017dc10786e9259d,"[0.19160914563647896, 0.043307168218026076, -0...","[0.0006823661944716316, -0.0001667280999103190...","[0.12353797170875343, -0.10053731489253163, 0....","[0.35214530199012284, -0.2733977116285107, -0....","[0.16655266271797456, -0.05843355822813081, -0...",395.500000,2,-1,-1
3,2c9207157e58b769017e61a3e13c0944,2c9207157e425d5c017e48061bb50e14,"[0.3352685938668621, -0.06193383868396666, 0.2...","[0.06824288797273276, 0.3345999082804966, -0.0...","[0.31934960678804675, 0.33476821858920475, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.21647338719008005, 0.6192388484570416, -0.0...",1583.000000,1,500,50
4,2c9207157e58b769017e9a416cff07d9,2c9207157e425d5c017e47121b3a6eef,"[0.193992358450605, -0.12803979466841925, -0.0...","[0.005677395289748753, 0.003916162454285459, 0...","[0.21814583275640784, -0.01721680318513782, -0...","[0.26224908552086346, 0.10211601798150097, 0.1...","[0.08803453540478255, 0.03652972730097373, 0.0...",639.250000,5,400,300
...,...,...,...,...,...,...,...,...,...,...,...
13581,8a69c4b5820f0b2101820f99257c07b7,2c9207158114fb2901811ee8a68f49ea,"[0.18894454111724257, 0.30492528922333145, 0.1...","[0.04997392570055781, -0.004502174307944112, 0...","[0.20130461498046187, -0.12255278588074696, 0....","[0.4372346675297544, -0.23536595083043993, -0....","[0.09944266033041682, 0.015459251512970253, -0...",534.333333,5,1200,670
13582,8a69d7a68219933d01823fff37d04ba3,8a69f6f8823a93b901823ffcfae57267,"[0.08540682135035549, 0.0464066016768764, 0.03...","[0.03691517113152803, -0.0035258117015806523, ...","[0.23004562348177082, -0.09744323796595142, 0....","[0.02841189931214894, 0.02501481787894325, 0.0...","[0.08820578402787298, -0.01840399298441212, -0...",NaN,2,-1,-1
13583,8a69d7c282199453018233eef4a43340,2c92071581ebf3390181f5b6628042bd,"[0.1557873230952529, 0.07473260131093742, -0.0...","[0.07266604344946537, -0.009615434260558572, 0...","[0.23905501196703516, -0.07952094007708325, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.12134680786849046, 0.0131295908164011, 0.03...",1353.500000,4,-1,180
13584,8a69d7c2821994530182395f8b7032c7,8a69f68e82198e91018234ee6739094a,"[0.1987571952893996, 0.07294446763122266, -0.1...","[0.271392628512948, -0.05242300248651601, 0.50...","[0.06003737476583179, -0.05225116621866782, -0...","[0.33075036193849955, -0.12686838157002917, 0....","[0.30693235404040736, -0.13829145445971983, -0...",502.000000,3,-1,-1


# new feature

In [10]:
# 保存为new feature
cvjd_lsa_work_duration_equal_job_salary.to_pickle('../generalization_data/test_new_feature_13586.pkl')


# raw feature

In [11]:
# 这是用来生成lsa向量的训练集，必须用这个数据集来拼接lsa向量
# 这份文件其实是用raw_cvjd的数据和train id进行join得到的
test_data_lsa_cvjd_13586 = pd.read_pickle('../generalization_data/test_data_lsa_cvjd_13586.pkl')
test_data_lsa_cvjd_13586

,cv_id,jd_id,jd_code,title,deadline,category_id,category_name,company_id,company_name,recruit_number,...,workYear,avatar,eduTracks,jobTracks,projectTracks,ds.1,da_label,rank_label,resume_type,order_month
1,2c9207157bf91042017c0d979cbb047d,2c9207157bcbb76b017bf7f2bbbb3153,JD20210918160449291911,会员管理经理,2021-10-31 23:59:59.0,130101,用户运营,2c9207157b9b9199017bc895b2f23a39,上海晨光文具股份有限公司,1,...,7.0,NaN,"['{""degree"": ""本科"", ""endDate"": ""2014-06"", ""id"":...","['{""companyName"": ""北京美餐好客科技有限公司"", ""companyNatu...","['{""description"": ""项目描述: 母婴类人群，包含童装、童鞋、奶粉。\\n运...",20220831,1,1,NaN,2021-09
2,2c9207157bf91042017c7232fbab3be1,2c9207157bf94d99017c108f769405e0,JD20210923104433723714,产品市场经理,2021-10-31 23:59:59.0,110101,产品经理,2c9207157bf94d99017c108765c804aa,凯斯纽荷兰（中国）管理有限公司,1,...,14.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2012-09"", ""id"":...","['{""companyName"": ""Continental Tires (China) L...",NaN,20220831,2,2,NaN,2021-10
9,2c9207157dbedf8d017dc7561a247ced,2c9207157dbedfdb017dc10786e9259d,JD20211216101338040534,专卖店店长,NaN,290304,门店店长,2c9207157b9b9199017b9f3db5430383,小米科技有限责任公司,1,...,6.0,NaN,"['{""degree"": ""大专"", ""endDate"": ""2015-06"", ""id"":...","['{""companyName"": ""广州金逸影视传媒股份有限公司"", ""descripti...","['{""companyName"": ""*霍尔果斯自由时光影院管理有限公司"", ""descri...",20220831,3,3,4.0,2022-03
15,2c9207157e58b769017e61a3e13c0944,2c9207157e425d5c017e48061bb50e14,JD20220111152353418578,Java开发工程师,NaN,100101,Java,2c9207157bf94d99017c2f902e952262,深圳纳欣科技有限公司,1,...,NaN,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2011-06"", ""id"":...","['{""companyName"": ""任职时间"", ""endDate"": ""9999-02""...","['{""description"": ""公司名称\\n盛彩科技信息服务有限公司\\n职位名称\...",20220831,1,1,NaN,2022-01
27,2c9207157e58b769017e9a416cff07d9,2c9207157e425d5c017e47121b3a6eef,JD20220111102941471259,APP主UI设计,NaN,120105,UI设计师,2c9207157dabb0c9017db32ad4b5182e,北京能量盒科技有限公司,1,...,8.0,NaN,"['{""degree"": ""本科"", ""endDate"": ""2014-07"", ""id"":...","['{""companyName"": ""北京默契破冰科技有限公司"", ""description...","['{""companyName"": ""*北京默契破冰科技有限公司"", ""descriptio...",20220831,1,1,4.0,2022-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82301,8a69c4b5820f0b2101820f99257c07b7,2c9207158114fb2901811ee8a68f49ea,JD20220531210148031570,人力负责人,NaN,150406,人力资源VP/CHO,2c920715800e886301805e59ca756861,马上消费金融股份有限公司,1,...,15.0,NaN,"['{""schoolTag"": ""985 211"", ""endDate"": ""2020-04...","['{""endDate"": ""2022-07"", ""companyName"": ""德坤供应链...","['{""endDate"": ""2020-08"", ""companyName"": ""德坤"", ...",20220831,1,1,NaN,2022-07
82314,8a69d7a68219933d01823fff37d04ba3,8a69f6f8823a93b901823ffcfae57267,JD20220208163227909229,合规总监,NaN,180302,法务,2c9207157ed86c8e017ed86eb428012b,东方财富信息股份有限公司,1,...,14.0,NaN,"['{""endDate"": ""2014-01"", ""unified"": true, ""deg...","['{""endDate"": ""2022-07"", ""companyName"": ""CNCB ...","['{""name"": null}']",20220831,2,2,4.0,2022-07
82318,8a69d7c282199453018233eef4a43340,2c92071581ebf3390181f5b6628042bd,JD20220713115939586417,交付经理,NaN,300701,其他生产制造职位,2c920715816798cb01816a305e95331b,广州视源电子科技股份有限公司,1,...,7.0,NaN,"['{""endDate"": ""2015-06"", ""unified"": true, ""deg...","['{""endDate"": ""2022-07"", ""companyName"": ""深南电路股...","['{""name"": null}']",20220831,3,3,4.0,2022-07
82319,8a69d7c2821994530182395f8b7032c7,8a69f68e82198e91018234ee6739094a,JD20220725183354121210,跨境电商运营经理,NaN,130124,跨境电商运营,2c9207158114fb29018132ace5f11fe9,广州视源电子科技股份有限公司,1,...,6.0,NaN,"['{""endDate"": ""2015-06"", ""unified"": true, ""deg...","['{""endDate"": ""2022-07"", ""companyName"": ""广州拉丁互...","['{""endDate"": ""2020-12"", ""jobTitle"": ""项目负责人"", ...",20220831,1,1,4.0,2022-07


In [12]:
# 这些特征在原始的cv和jd表里
raw_columns = ['age', 'recruit_number', 'workYear', 'min_month_salary', 'max_month_salary', 'pay_months']

# 将raw特征保存
test_data_lsa_cvjd_13586[['cv_id', 'jd_id'] + raw_columns].to_pickle('../generalization_data/test_raw_feature_13586.pkl')


# ext特征

In [13]:
# 这些特征在sample pipeline的表里
ext_columns = ['school_labels_cv', 'school_labels_jd', 'expectlocation_range_cv', 'location_range_jd', 'degree_index_cv', 'degree_index_jd', 'workyear_range_jd', 'gender_index_cv', 'job_intentstatus_cv']


In [14]:
# 读取包含label的sample表
all_sample = pd.read_csv('../data_20220831/sample_20220831_spark.csv').drop('Unnamed: 0', axis=1).drop_duplicates(subset=['cv_id', 'jd_id'], keep='first').reset_index(drop=True)

test_path = '../generalization_data/cvjd_test_filter_13586.pkl'
test_id = pd.read_pickle(test_path)[['cv_id', 'jd_id', 'order_month']]

all_sample = all_sample.join(test_id.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner')
all_sample

/Users/jinrirencai/opt/anaconda3/envs/tsinghua/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,cv_id,jd_id,cv_code,jd_code,last_opt,da_label,rank_label,title,deadline,category_id,...,industry_jd2vec,industry_vec2,majorName_cut,majorName_jd2vec,majorName_vec2,skills_cut,skills_jd2vec,skills_vec2,ds,order_month
3,2c9207157bf91042017c62fce2583411,2c9207157b9b9199017bca11fd3d3f11,C972256920557719589,JD20210908152800055103,reject,2,2,高级用户运营专家,2021-12-31 23:59:59.0,130405,...,"[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....",['市场营销'],"[0.6256594657897949, -0.07856443524360657, 0.0...","[0.7786060571670532, -0.8168095946311951, -0.6...","['[', '""', 'Vi', '""', ',', '""', 'Pr', '""', ','...","[0.3983086943626404, 0.11806842684745789, -0.2...","[0.29188045859336853, -0.04354493319988251, -0...",20220831,2021-10
14,2c9207157dec317f017e0aaa31f13c30,2c9207157ccbcd86017cdb5ab2e80261,C972256921644044690,JD20211101195140678080,drop,1,1,团购城市经理,2022-05-31 23:59:59.0,140000,...,"[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....",['工商管理'],"[0.2706994414329529, -0.26152682304382324, 0.0...","[0.32410022616386414, -1.0074623823165894, -0....","['[', '""', 'App', '""', ',', '""', 'View', '""', ...","[0.3733346462249756, 0.1439696103334427, -0.23...","[0.16608959436416626, -0.01653897576034069, -0...",20220831,2021-12
19,2c9207157e58b769017e6b473d056f15,2c9207157dec2c34017e059809471ddf,C972256920608051330,JD20211223165937570608,drop,1,1,Innovation sales manager,NaN,140315,...,"[-0.2878880202770233, 0.15149953961372375, 0.5...","[-0.5253047943115234, 1.0256906747817993, -0.6...","['热能', '与', '动力工程']","[-0.22125621140003204, 0.2544739246368408, -0....","[-0.011675124056637287, 0.38259899616241455, -...","['#', 'null']","[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....",20220831,2022-01
27,2c9207157e58b769017e7700fa3a6945,2c9207157d710969017d7f001a9b0b26,C972256920616440167,JD20211203142530792573,drop,1,1,硬件产品经理,NaN,110000,...,"[0.11452177166938782, -0.0988738089799881, -0....","[0.825724720954895, 0.18566258251667023, -0.24...","['能源', '与', '动力工程']","[-0.1743580549955368, 0.17508043348789215, -0....","[-0.15169839560985565, 0.23648810386657715, -0...","['[', '""', 'c', '端', '产品', '""', ',', '""', 'b',...","[0.3175760507583618, 0.06867434084415436, -0.2...","[0.4253033697605133, -0.0385710708796978, -0.2...",20220831,2022-01
32,2c9207157e9b5f31017ed39a81906791,2c9207157dc19933017dc1c0938a0423,C972256920633217232,JD20211216133923089857,NaN,0,0,并购财务总监,NaN,150308,...,"[0.27394238114356995, -0.04573575034737587, -0...","[0.3191254138946533, -0.06701976805925369, -0....",['财务管理'],"[0.45438629388809204, -0.08840988576412201, 0....","[0.18450282514095306, -1.0731631517410278, -1....","['[', '""', '财务管理', '""', ',', '""', '融资', '管理', ...","[0.2799074649810791, 0.10806544870138168, -0.1...","[0.25339335203170776, 0.131011962890625, -0.34...",20220831,2022-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82296,2c92071581d380c20181fbbb8c003c9f,2c920715816798cb018170d38e595d74,C997171369823309824,JD20220617101341895033,reject,2,2,AI Research Scientist – Computational design a...,NaN,101101,...,"[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....","['河北', '工业', '大学']","[0.24242587387561798, -0.44595611095428467, 0....","[0.37358707189559937, 0.6225378513336182, -0.1...","['Ai', ',', 'Pc', ',', 'Ar', ',', 'Ui', ',', '...","[0.4520522654056549, -0.006344459485262632, -0...","[0.17360220849514008, 0.01891912892460823, -0....",20220831,2022-07
82304,8a69c4b4820f0b2101820f345bf826fc,2c92071581c8c61e0181cd2af3a41fc1,C998541584704344064,JD20220607151404791589,drop,1,1,NFT产品经理,NaN,110101,...,"[0.1434544324874878, -0.17350050806999207, -0....","[0.7683403491973877, -0.29662415385246277, -0....",['哈尔滨工业大学'],"[-0.09854261577129364, -0.4327467978000641, -0...","[-0.138335183262825, -0.2094516158103943, -0.4...","['PPT', '

In [15]:
test_id.columns

Index(['cv_id', 'jd_id', 'order_month'], dtype='object')

In [16]:
# 将ext特征保存
all_sample[['cv_id', 'jd_id'] + ext_columns].to_pickle('../generalization_data/test_ext_feature_13586.pkl')

# 拼接new raw ext特征

In [17]:
base_path = '../generalization_data/'
raw_feature_file = 'test_raw_feature_13586.pkl'
ext_feature_file = 'test_ext_feature_13586.pkl'
new_feature_file = 'test_new_feature_13586.pkl'

# 读取特征文件
raw_feature = pd.read_pickle(base_path + raw_feature_file)
ext_feature = pd.read_pickle(base_path + ext_feature_file)
new_feature = pd.read_pickle(base_path + new_feature_file)


In [18]:
new_feature

,cv_id,jd_id,test_description_tfidf_lsa,test_position_tfidf_lsa,test_requirement_tfidf_lsa,test_skills_tfidf_lsa,test_title_category_tags_tfidf_lsa,work_duration_mean,equal_job,parsed_desiredSalary,parsed_currentSalary
0,2c9207157bf91042017c0d979cbb047d,2c9207157bcbb76b017bf7f2bbbb3153,"[0.17896497513724188, 0.07571414550659793, -0....","[0.171817730230271, -0.035070685612599815, -0....","[0.04458234091145562, -0.03407758459309192, 0....","[0.3582266403278269, -0.17046381930752355, 0.2...","[0.3252484303865592, -0.1404601755384386, -0.1...",506.000000,4,-1,-1
1,2c9207157bf91042017c7232fbab3be1,2c9207157bf94d99017c108f769405e0,"[0.0022543548572903855, 0.0013497502728216206,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[7.362506665850695e-05, -1.7992806189079e-05, ...","[0.025060678965842003, 0.008574558405555977, -...","[0.3068414446421549, -0.0731466854560243, 0.34...",811.500000,0,-1,-1
2,2c9207157dbedf8d017dc7561a247ced,2c9207157dbedfdb017dc10786e9259d,"[0.19160914563647896, 0.043307168218026076, -0...","[0.0006823661944716316, -0.0001667280999103190...","[0.12353797170875343, -0.10053731489253163, 0....","[0.35214530199012284, -0.2733977116285107, -0....","[0.16655266271797456, -0.05843355822813081, -0...",395.500000,2,-1,-1
3,2c9207157e58b769017e61a3e13c0944,2c9207157e425d5c017e48061bb50e14,"[0.3352685938668621, -0.06193383868396666, 0.2...","[0.06824288797273276, 0.3345999082804966, -0.0...","[0.31934960678804675, 0.33476821858920475, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.21647338719008005, 0.6192388484570416, -0.0...",1583.000000,1,500,50
4,2c9207157e58b769017e9a416cff07d9,2c9207157e425d5c017e47121b3a6eef,"[0.193992358450605, -0.12803979466841925, -0.0...","[0.005677395289748753, 0.003916162454285459, 0...","[0.21814583275640784, -0.01721680318513782, -0...","[0.26224908552086346, 0.10211601798150097, 0.1...","[0.08803453540478255, 0.03652972730097373, 0.0...",639.250000,5,400,300
...,...,...,...,...,...,...,...,...,...,...,...
13581,8a69c4b5820f0b2101820f99257c07b7,2c9207158114fb2901811ee8a68f49ea,"[0.18894454111724257, 0.30492528922333145, 0.1...","[0.04997392570055781, -0.004502174307944112, 0...","[0.20130461498046187, -0.12255278588074696, 0....","[0.4372346675297544, -0.23536595083043993, -0....","[0.09944266033041682, 0.015459251512970253, -0...",534.333333,5,1200,670
13582,8a69d7a68219933d01823fff37d04ba3,8a69f6f8823a93b901823ffcfae57267,"[0.08540682135035549, 0.0464066016768764, 0.03...","[0.03691517113152803, -0.0035258117015806523, ...","[0.23004562348177082, -0.09744323796595142, 0....","[0.02841189931214894, 0.02501481787894325, 0.0...","[0.08820578402787298, -0.01840399298441212, -0...",NaN,2,-1,-1
13583,8a69d7c282199453018233eef4a43340,2c92071581ebf3390181f5b6628042bd,"[0.1557873230952529, 0.07473260131093742, -0.0...","[0.07266604344946537, -0.009615434260558572, 0...","[0.23905501196703516, -0.07952094007708325, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.12134680786849046, 0.0131295908164011, 0.03...",1353.500000,4,-1,180
13584,8a69d7c2821994530182395f8b7032c7,8a69f68e82198e91018234ee6739094a,"[0.1987571952893996, 0.07294446763122266, -0.1...","[0.271392628512948, -0.05242300248651601, 0.50...","[0.06003737476583179, -0.05225116621866782, -0...","[0.33075036193849955, -0.12686838157002917, 0....","[0.30693235404040736, -0.13829145445971983, -0...",502.000000,3,-1,-1


In [19]:
# 3份特征文件拼接
medium = raw_feature.join(ext_feature.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner')
raw_ext_new_feature = medium.join(new_feature.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner')
raw_ext_new_feature

,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,job_intentstatus_cv,test_description_tfidf_lsa,test_position_tfidf_lsa,test_requirement_tfidf_lsa,test_skills_tfidf_lsa,test_title_category_tags_tfidf_lsa,work_duration_mean,equal_job,parsed_desiredSalary,parsed_currentSalary
1,2c9207157bf91042017c0d979cbb047d,2c9207157bcbb76b017bf7f2bbbb3153,29.0,1,7.0,NaN,NaN,NaN,[],[],...,2,"[0.17896497513724188, 0.07571414550659793, -0....","[0.171817730230271, -0.035070685612599815, -0....","[0.04458234091145562, -0.03407758459309192, 0....","[0.3582266403278269, -0.17046381930752355, 0.2...","[0.3252484303865592, -0.1404601755384386, -0.1...",506.000000,4,-1,-1
2,2c9207157bf91042017c7232fbab3be1,2c9207157bf94d99017c108f769405e0,NaN,1,14.0,NaN,NaN,NaN,[],[],...,3,"[0.0022543548572903855, 0.0013497502728216206,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[7.362506665850695e-05, -1.7992806189079e-05, ...","[0.025060678965842003, 0.008574558405555977, -...","[0.3068414446421549, -0.0731466854560243, 0.34...",811.500000,0,-1,-1
9,2c9207157dbedf8d017dc7561a247ced,2c9207157dbedfdb017dc10786e9259d,27.0,1,6.0,NaN,NaN,NaN,['双一流'],[],...,3,"[0.19160914563647896, 0.043307168218026076, -0...","[0.0006823661944716316, -0.0001667280999103190...","[0.12353797170875343, -0.10053731489253163, 0....","[0.35214530199012284, -0.2733977116285107, -0....","[0.16655266271797456, -0.05843355822813081, -0...",395.500000,2,-1,-1
15,2c9207157e58b769017e61a3e13c0944,2c9207157e425d5c017e48061bb50e14,38.0,1,NaN,25.0,35.0,15.0,[],['双一流'],...,3,"[0.3352685938668621, -0.06193383868396666, 0.2...","[0.06824288797273276, 0.3345999082804966, -0.0...","[0.31934960678804675, 0.33476821858920475, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.21647338719008005, 0.6192388484570416, -0.0...",1583.000000,1,500,50
27,2c9207157e58b769017e9a416cff07d9,2c9207157e425d5c017e47121b3a6eef,26.0,1,8.0,25.0,45.0,15.0,[],"['985', '211', '双一流']",...,3,"[0.193992358450605, -0.12803979466841925, -0.0...","[0.005677395289748753, 0.003916162454285459, 0...","[0.21814583275640784, -0.01721680318513782, -0...","[0.26224908552086346, 0.10211601798150097, 0.1...","[0.08803453540478255, 0.03652972730097373, 0.0...",639.250000,5,400,300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82301,8a69c4b5820f0b2101820f99257c07b7,2c9207158114fb2901811ee8a68f49ea,37.0,1,15.0,170.0,250.0,12.0,"['985', '211', '双一流']",[],...,2,"[0.18894454111724257, 0.30492528922333145, 0.1...","[0.04997392570055781, -0.004502174307944112, 0...","[0.20130461498046187, -0.12255278588074696, 0....","[0.4372346675297544, -0.23536595083043993, -0....","[0.09944266033041682, 0.015459251512970253, -0...",534.333333,5,1200,670
82314,8a69d7a68219933d01823fff37d04ba3,8a69f6f8823a93b901823ffcfae57267,40.0,1,14.0,60.0,80.0,16.0,[],[],...,2,"[0.08540682135035549, 0.0464066016768764, 0.03...","[0.03691517113152803, -0.0035258117015806523, ...","[0.23004562348177082, -0.09744323796595142, 0....","[0.02841189931214894, 0.02501481787894325, 0.0...","[0.08820578402787298, -0.01840399298441212, -0...",NaN,2,-1,-1
82318,8a69d7c282199453018233eef4a43340,2c92071581ebf3390181f5b6628042bd,29.0,1,7.0,20.0,25.0,12.0,[],[],...,2,"[0.1557873230952529, 0.07473260131093742, -0.0...","[0.07266604344946537, -0.009615434260558572, 0...","[0.23905501196703516, -0.07952094007708325, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.12134680786849046, 0.0131295908164011, 0.03...",1353.500000,4,-1,180
82319,8a69d7c2821994530182395f8b7032c7,8a69f68e82198e91018234ee6739094a,30.0,1,6.0,15.0,25.0,13.0,[],[],...,2,"[0.1987571952893996, 0.07294446763122266, -0.1...","[0.271392628512948, -0.05242300248651601, 0.50...","[0.06003737476583179, -0.05225116621866782, -0...","[0.33075036193849955, -0.12686838157002917, 0....","[0.30693235404040736, -0.13829145445971983, -0...",502.000000,3,-1,-1


In [20]:
# 读取包含label的sample表
all_sample = pd.read_csv('../data_20220831/sample_20220831_spark.csv').drop('Unnamed: 0', axis=1).drop_duplicates(subset=['cv_id', 'jd_id'], keep='first').reset_index(drop=True)

test_path = '../generalization_data/cvjd_test_filter_13586.pkl'
test_id = pd.read_pickle(test_path)[['cv_id', 'jd_id', 'order_month']]

all_sample = all_sample.join(test_id.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner')
all_sample

/Users/jinrirencai/opt/anaconda3/envs/tsinghua/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,cv_id,jd_id,cv_code,jd_code,last_opt,da_label,rank_label,title,deadline,category_id,...,industry_jd2vec,industry_vec2,majorName_cut,majorName_jd2vec,majorName_vec2,skills_cut,skills_jd2vec,skills_vec2,ds,order_month
3,2c9207157bf91042017c62fce2583411,2c9207157b9b9199017bca11fd3d3f11,C972256920557719589,JD20210908152800055103,reject,2,2,高级用户运营专家,2021-12-31 23:59:59.0,130405,...,"[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....",['市场营销'],"[0.6256594657897949, -0.07856443524360657, 0.0...","[0.7786060571670532, -0.8168095946311951, -0.6...","['[', '""', 'Vi', '""', ',', '""', 'Pr', '""', ','...","[0.3983086943626404, 0.11806842684745789, -0.2...","[0.29188045859336853, -0.04354493319988251, -0...",20220831,2021-10
14,2c9207157dec317f017e0aaa31f13c30,2c9207157ccbcd86017cdb5ab2e80261,C972256921644044690,JD20211101195140678080,drop,1,1,团购城市经理,2022-05-31 23:59:59.0,140000,...,"[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....",['工商管理'],"[0.2706994414329529, -0.26152682304382324, 0.0...","[0.32410022616386414, -1.0074623823165894, -0....","['[', '""', 'App', '""', ',', '""', 'View', '""', ...","[0.3733346462249756, 0.1439696103334427, -0.23...","[0.16608959436416626, -0.01653897576034069, -0...",20220831,2021-12
19,2c9207157e58b769017e6b473d056f15,2c9207157dec2c34017e059809471ddf,C972256920608051330,JD20211223165937570608,drop,1,1,Innovation sales manager,NaN,140315,...,"[-0.2878880202770233, 0.15149953961372375, 0.5...","[-0.5253047943115234, 1.0256906747817993, -0.6...","['热能', '与', '动力工程']","[-0.22125621140003204, 0.2544739246368408, -0....","[-0.011675124056637287, 0.38259899616241455, -...","['#', 'null']","[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....",20220831,2022-01
27,2c9207157e58b769017e7700fa3a6945,2c9207157d710969017d7f001a9b0b26,C972256920616440167,JD20211203142530792573,drop,1,1,硬件产品经理,NaN,110000,...,"[0.11452177166938782, -0.0988738089799881, -0....","[0.825724720954895, 0.18566258251667023, -0.24...","['能源', '与', '动力工程']","[-0.1743580549955368, 0.17508043348789215, -0....","[-0.15169839560985565, 0.23648810386657715, -0...","['[', '""', 'c', '端', '产品', '""', ',', '""', 'b',...","[0.3175760507583618, 0.06867434084415436, -0.2...","[0.4253033697605133, -0.0385710708796978, -0.2...",20220831,2022-01
32,2c9207157e9b5f31017ed39a81906791,2c9207157dc19933017dc1c0938a0423,C972256920633217232,JD20211216133923089857,NaN,0,0,并购财务总监,NaN,150308,...,"[0.27394238114356995, -0.04573575034737587, -0...","[0.3191254138946533, -0.06701976805925369, -0....",['财务管理'],"[0.45438629388809204, -0.08840988576412201, 0....","[0.18450282514095306, -1.0731631517410278, -1....","['[', '""', '财务管理', '""', ',', '""', '融资', '管理', ...","[0.2799074649810791, 0.10806544870138168, -0.1...","[0.25339335203170776, 0.131011962890625, -0.34...",20220831,2022-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82296,2c92071581d380c20181fbbb8c003c9f,2c920715816798cb018170d38e595d74,C997171369823309824,JD20220617101341895033,reject,2,2,AI Research Scientist – Computational design a...,NaN,101101,...,"[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....","['河北', '工业', '大学']","[0.24242587387561798, -0.44595611095428467, 0....","[0.37358707189559937, 0.6225378513336182, -0.1...","['Ai', ',', 'Pc', ',', 'Ar', ',', 'Ui', ',', '...","[0.4520522654056549, -0.006344459485262632, -0...","[0.17360220849514008, 0.01891912892460823, -0....",20220831,2022-07
82304,8a69c4b4820f0b2101820f345bf826fc,2c92071581c8c61e0181cd2af3a41fc1,C998541584704344064,JD20220607151404791589,drop,1,1,NFT产品经理,NaN,110101,...,"[0.1434544324874878, -0.17350050806999207, -0....","[0.7683403491973877, -0.29662415385246277, -0....",['哈尔滨工业大学'],"[-0.09854261577129364, -0.4327467978000641, -0...","[-0.138335183262825, -0.2094516158103943, -0.4...","['PPT', '

In [21]:
# 拼接特征和label并保存
label_df = all_sample[['cv_id', 'jd_id', 'da_label', 'rank_label']]
raw_ext_new_feature_label = raw_ext_new_feature.join(label_df.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner')
raw_ext_new_feature_label.to_pickle('../generalization_data/test_raw_ext_new_feature_label_13586.pkl')

In [22]:
raw_ext_new_feature_label

,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,test_position_tfidf_lsa,test_requirement_tfidf_lsa,test_skills_tfidf_lsa,test_title_category_tags_tfidf_lsa,work_duration_mean,equal_job,parsed_desiredSalary,parsed_currentSalary,da_label,rank_label
1,2c9207157bf91042017c0d979cbb047d,2c9207157bcbb76b017bf7f2bbbb3153,29.0,1,7.0,NaN,NaN,NaN,[],[],...,"[0.171817730230271, -0.035070685612599815, -0....","[0.04458234091145562, -0.03407758459309192, 0....","[0.3582266403278269, -0.17046381930752355, 0.2...","[0.3252484303865592, -0.1404601755384386, -0.1...",506.000000,4,-1,-1,1,1
2,2c9207157bf91042017c7232fbab3be1,2c9207157bf94d99017c108f769405e0,NaN,1,14.0,NaN,NaN,NaN,[],[],...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[7.362506665850695e-05, -1.7992806189079e-05, ...","[0.025060678965842003, 0.008574558405555977, -...","[0.3068414446421549, -0.0731466854560243, 0.34...",811.500000,0,-1,-1,2,2
9,2c9207157dbedf8d017dc7561a247ced,2c9207157dbedfdb017dc10786e9259d,27.0,1,6.0,NaN,NaN,NaN,['双一流'],[],...,"[0.0006823661944716316, -0.0001667280999103190...","[0.12353797170875343, -0.10053731489253163, 0....","[0.35214530199012284, -0.2733977116285107, -0....","[0.16655266271797456, -0.05843355822813081, -0...",395.500000,2,-1,-1,3,3
15,2c9207157e58b769017e61a3e13c0944,2c9207157e425d5c017e48061bb50e14,38.0,1,NaN,25.0,35.0,15.0,[],['双一流'],...,"[0.06824288797273276, 0.3345999082804966, -0.0...","[0.31934960678804675, 0.33476821858920475, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.21647338719008005, 0.6192388484570416, -0.0...",1583.000000,1,500,50,1,1
27,2c9207157e58b769017e9a416cff07d9,2c9207157e425d5c017e47121b3a6eef,26.0,1,8.0,25.0,45.0,15.0,[],"['985', '211', '双一流']",...,"[0.005677395289748753, 0.003916162454285459, 0...","[0.21814583275640784, -0.01721680318513782, -0...","[0.26224908552086346, 0.10211601798150097, 0.1...","[0.08803453540478255, 0.03652972730097373, 0.0...",639.250000,5,400,300,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82301,8a69c4b5820f0b2101820f99257c07b7,2c9207158114fb2901811ee8a68f49ea,37.0,1,15.0,170.0,250.0,12.0,"['985', '211', '双一流']",[],...,"[0.04997392570055781, -0.004502174307944112, 0...","[0.20130461498046187, -0.12255278588074696, 0....","[0.4372346675297544, -0.23536595083043993, -0....","[0.09944266033041682, 0.015459251512970253, -0...",534.333333,5,1200,670,1,1
82314,8a69d7a68219933d01823fff37d04ba3,8a69f6f8823a93b901823ffcfae57267,40.0,1,14.0,60.0,80.0,16.0,[],[],...,"[0.03691517113152803, -0.0035258117015806523, ...","[0.23004562348177082, -0.09744323796595142, 0....","[0.02841189931214894, 0.02501481787894325, 0.0...","[0.08820578402787298, -0.01840399298441212, -0...",NaN,2,-1,-1,2,2
82318,8a69d7c282199453018233eef4a43340,2c92071581ebf3390181f5b6628042bd,29.0,1,7.0,20.0,25.0,12.0,[],[],...,"[0.07266604344946537, -0.009615434260558572, 0...","[0.23905501196703516, -0.07952094007708325, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.12134680786849046, 0.0131295908164011, 0.03...",1353.500000,4,-1,180,3,3
82319,8a69d7c2821994530182395f8b7032c7,8a69f68e82198e91018234ee6739094a,30.0,1,6.0,15.0,25.0,13.0,[],[],...,"[0.271392628512948, -0.05242300248651601, 0.50...","[0.06003737476583179, -0.05225116621866782, -0...","[0.33075036193849955, -0.12686838157002917, 0....","[0.30693235404040736, -0.13829145445971983, -0...",502.000000,3,-1,-1,1,1


In [23]:
sample_test = raw_ext_new_feature_label

In [24]:
sample_test.columns

Index(['cv_id', 'jd_id', 'age', 'recruit_number', 'workYear',
       'min_month_salary', 'max_month_salary', 'pay_months',
       'school_labels_cv', 'school_labels_jd', 'expectlocation_range_cv',
       'location_range_jd', 'degree_index_cv', 'degree_index_jd',
       'workyear_range_jd', 'gender_index_cv', 'job_intentstatus_cv',
       'test_description_tfidf_lsa', 'test_position_tfidf_lsa',
       'test_requirement_tfidf_lsa', 'test_skills_tfidf_lsa',
       'test_title_category_tags_tfidf_lsa', 'work_duration_mean', 'equal_job',
       'parsed_desiredSalary', 'parsed_currentSalary', 'da_label',
       'rank_label'],
      dtype='object')

In [25]:
!pwd

/Users/jinrirencai/Desktop/golden_data_from_github/generalization


In [ ]:
# 中位数填充
import pickle

imp_pkl_filename = 'age_missing_model.pkl'
with open(imp_pkl_filename, 'rb') as file:
    imp = pickle.load(file)
sample_test['age'] = imp.transform(sample_test[['age']])

imp_pkl_filename = 'workYear_missing_model.pkl'
with open(imp_pkl_filename, 'rb') as file:
    imp = pickle.load(file)
sample_test['workYear'] = imp.transform(sample_test[['workYear']])

imp_pkl_filename = 'min_month_salary_missing_model.pkl'
with open(imp_pkl_filename, 'rb') as file:
    imp = pickle.load(file)
sample_test['min_month_salary'] = imp.transform(sample_test[['min_month_salary']])

imp_pkl_filename = 'max_month_salary_missing_model.pkl'
with open(imp_pkl_filename, 'rb') as file:
    imp = pickle.load(file)
sample_test['max_month_salary'] = imp.transform(sample_test[['max_month_salary']])

imp_pkl_filename = 'pay_months_missing_model.pkl'
with open(imp_pkl_filename, 'rb') as file:
    imp = pickle.load(file)
sample_test['pay_months'] = imp.transform(sample_test[['pay_months']])

imp_pkl_filename = 'work_duration_mean_missing_model.pkl'
with open(imp_pkl_filename, 'rb') as file:
    imp = pickle.load(file)
sample_test['work_duration_mean'] = imp.transform(sample_test[['work_duration_mean']])





# 这两个只能选一个接入模型
imp_pkl_filename = 'parsed_desiredSalary_missing_model.pkl'
with open(imp_pkl_filename, 'rb') as file:
    imp = pickle.load(file)
sample_test['parsed_desiredSalary'] = imp.transform(sample_test[['parsed_desiredSalary']])

imp_pkl_filename = 'parsed_currentSalary_missing_model.pkl'
with open(imp_pkl_filename, 'rb') as file:
    imp = pickle.load(file)
sample_test['parsed_currentSalary'] = imp.transform(sample_test[['parsed_currentSalary']])


sample_test
